In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.data_pipeline.mongodb_manager import MongoDBManager
from iris.data_pipeline.image_store_manager import ImageStoreManager
from iris.data_pipeline.product_handler import ProductHandler
from iris.data_pipeline.web_shop_scraper import WebShopScraper


In [ ]:
# Initialize configuration manager
config_manager = DataPipelineConfigManager() 
shop_config = config_manager.shop_config
image_store_config = config_manager.image_store_config
mongodb_config = config_manager.mongodb_config

# Create MongoDB manager with configuration
mongodb_manager = MongoDBManager(mongodb_config)
image_store_manager = ImageStoreManager(image_store_config)

# Initialize 
web_shop_scraper = WebShopScraper(
    shop_config=shop_config,
    product_handler=ProductHandler(shop_config=shop_config),
)

# Start scraping
with mongodb_manager as db:
    for product, images in web_shop_scraper.scrape():
        print(f"Scraped product: {product.data['title']}, Images: {len(images)}")

        for image in images:
            image.load_image(image_store_manager)

        db.upsert(db.config.product_collection, product)
        db.upsert(db.config.image_metadata_collection, images)
